In [1]:
import json
import gensim
from nltk.corpus import stopwords
import nltk
import numpy as np
import os

In [2]:
#fetch all reviews
data=[]
for root, dirs, files in os.walk('../Reviews/'):
    for name in dirs:                             #all review folders
        with open('../Reviews/'+name+'/review.txt') as f:
            data=data+json.load(f)
            f.close()
dataset=[]
dataset=np.asarray(data)
dataset=np.concatenate(dataset)               #contains all the reviews

In [3]:
def remove_stopwords(tokens):                                # for removing stopwords
    stop=set(stopwords.words('english'))
    stopwords_tokens=[token for token in tokens if token not in stop]
    return stopwords_tokens

In [4]:
def tokenize_sent(sentence):                         # for tokenizing single sentence
    temp_tokens=nltk.word_tokenize(sentence)
    sent_tokens=remove_stopwords(temp_tokens)
    return sent_tokens

In [5]:
def tokenize_review(review):                # for tokenizing single reviews
    review=review.lower()
    sentences=nltk.sent_tokenize(review)
    review_tokens=[]
    for sentence in sentences:
        review_tokens.append(tokenize_sent(sentence))
    return review_tokens

In [8]:
def tokenize_all(dataset):               # for tokenizing all the reviews
    tokens=[]
    for review in dataset:
        tokens=tokens+tokenize_review(review)
    return tokens    

In [29]:
#'tokens' contain all the tokens
tokens=tokenize_all(dataset)

In [57]:
#storing entire tokens to a a file
with open('tokens.txt','w') as f:
    json.dump(tokens,f)     

In [58]:
len(tokens)

249932

In [68]:
# training word2vec
model = gensim.models.Word2Vec(tokens, min_count=5, workers=2)
model.save('model')              #saves the model

In [72]:
#testing most_similar
#model.most_similar('work')

In [65]:
#testing similarity
#model.similarity('job','work')

In [71]:
# testing load feature
#new_model=gensim.models.Word2Vec.load('model')
#new_model.most_similar('work')